# Tensorflow MNIST-LeNet Patch Demo

This notebook demonstrates the adversarial patch attack applied on the LeNet model, as well as preprocessing and adversarial training defenses.

The following two sections cover experiment setup and is similar across all demos.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [6]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_adversarial_patches"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The training and testing images in this directory are saved as PNG files and are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `Mnist/training/` and `Mnist/testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [78]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar czf workflows.tar.gz src/gaussian_augmentation.py src/deploy_patch.py src/init_model.py src/defenses_image_preprocessing.py src/tasks.py src/train.py src/import_keras.py src/spatial_smoothing.py src/generate_patch.py src/jpeg_compression.py src/attacks_patch_updated.py src/fgm.py src/infer.py src/registry_art_updated.py src/estimators_keras_classifiers_updated.py src/data_tensorflow_updated.py MLproject
chmod 644 workflows.tar.gz


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [8]:
restapi_client = utils.SecuringAIClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [9]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 10,
 'name': 'howard_mnist_adversarial_patches',
 'lastModified': '2020-11-05T08:45:14.946823',
 'createdOn': '2020-11-05T08:45:14.946823'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [10]:
restapi_client.list_queues()

[{'name': 'tensorflow_cpu',
  'lastModified': '2020-11-20T17:46:06.756687',
  'createdOn': '2020-11-20T17:46:06.756687',
  'queueId': 1},
 {'name': 'tensorflow_gpu',
  'lastModified': '2020-11-20T18:00:40.876888',
  'createdOn': '2020-11-20T18:00:40.876888',
  'queueId': 2},
 {'name': 'pytorch_cpu',
  'lastModified': '2020-11-20T19:52:36.079781',
  'createdOn': '2020-11-20T19:52:36.079781',
  'queueId': 5},
 {'name': 'pytorch_gpu',
  'lastModified': '2020-11-20T19:52:43.348460',
  'createdOn': '2020-11-20T19:52:43.348460',
  'queueId': 7}]

## Adversarial Patches: Baseline MNIST Training

Now, we will train our baseline LeNet-5 model on the MNIST dataset. 
We will be submitting our jobs to the `"tensorflow_gpu"` queue.
Once the experiment is finished, we will examine the accuracy results of our model.

In [17]:
# Create and submit training job.
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-04-28T19:32:34.141211',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_adversarial_patches_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 10,
 'jobId': '1576315b-5b62-494b-a195-68f39e991052',
 'lastModified': '2021-04-28T19:32:34.141211',
 'mlflowRunId': None,
 'queueId': 1,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/4c7faddada4c452b83da176be5f3d0a3/workflows.tar.gz'}


The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [18]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response

Now wait for the job to complete before proceeding to next steps.

In [19]:
response_le_net_train = wait_until_finished(response_le_net_train)
print("Training job for LeNet-5 neural network")
pprint.pprint(response_le_net_train)

Training job for LeNet-5 neural network
{'createdOn': '2021-04-28T19:32:34.141211',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_adversarial_patches_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 10,
 'jobId': '1576315b-5b62-494b-a195-68f39e991052',
 'lastModified': '2021-04-28T19:43:43.778764',
 'mlflowRunId': '09649012f75049e999eb085b0e7e1011',
 'queueId': 1,
 'status': 'finished',
 'timeout': '1h',
 'workflowUri': 's3://workflow/4c7faddada4c452b83da176be5f3d0a3/workflows.tar.gz'}


### Checking baseline MNIST job accuracy

Once the job has finished running we can view the results either through the MLflow URI or by accessing the job via MLflow client.
Here we will show the baseline accuracy results from the previous training job.
Please see [Querying the MLFlow Tracking Service](#Querying-the-MLFlow-Tracking-Service) section for more details.

In [20]:
# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


results = get_mlflow_results(response_le_net_train)
pprint.pprint(results.data.metrics)

{'accuracy': 0.9871052503585815,
 'auc': 0.999603271484375,
 'loss': 0.039359905428823734,
 'precision': 0.9889448285102844,
 'recall': 0.985792875289917,
 'restored_epoch': 6.0,
 'stopped_epoch': 11.0,
 'training_time_in_minutes': 10.7986485,
 'val_accuracy': 0.9886628985404968,
 'val_auc': 0.9993653893470764,
 'val_loss': 0.03706372629335903,
 'val_precision': 0.9896459579467773,
 'val_recall': 0.987995982170105}


With the baseline training settings, it appears that the LeNet model has been properly trained on MNIST dataset.

### Deploying and Testing Adversarial Patches

Now we will create and apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.
We will also apply the patches over the training set for the adversarial training defense evaluation.

#### Patch Generation

The following job will generate the adversarial patches. 
Feel free to adjust the input parameters to see how they impact the effectiveness of the patch attack.

In [80]:
# Create Patches
response_le_net_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            f"-P data_dir={DATASET_DIR}/training",
            "-P num_patch_gen_samples=40",
            "-P num_patch=3",
            "-P patch_target=5",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)


print("Patch attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_le_net_patches)
print("")

# Wait for Patch attack to finish.
response_le_net_patches = wait_until_finished(response_le_net_patches)

Patch attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-04-29T03:20:55.129645',
 'dependsOn': '1576315b-5b62-494b-a195-68f39e991052',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/training '
                     '-P num_patch_gen_samples=40 -P num_patch=3 -P '
                     'patch_target=5',
 'experimentId': 10,
 'jobId': '39a44294-3b7e-4fa2-b4f1-3e8ed026884e',
 'lastModified': '2021-04-29T03:20:55.129645',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/eb5af3c83940458c98996574a9ed16c5/workflows.tar.gz'}



#### Patch Deployment

The following jobs will deploy the patches over the training and test sets.

In [103]:
# Deploy Patch attack on training set.

response_deploy_le_net_patches_mnist_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            f"-P data_dir={DATASET_DIR}/training",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_training)
print("")

# Get the run ID of the test set.
response_deploy_le_net_patches_mnist_training = get_run_id(response_deploy_le_net_patches_mnist_training)

Patch deployment (LeNet-5 architecture) job submitted

{'createdOn': '2021-04-29T04:16:15.677833',
 'dependsOn': '39a44294-3b7e-4fa2-b4f1-3e8ed026884e',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=b81c3b9453e24dc8850d074b43c56b93 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/training',
 'experimentId': 10,
 'jobId': 'dd0406ef-01c1-4675-aba9-6c8e2965f0d6',
 'lastModified': '2021-04-29T04:16:15.677833',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/d4f00680b5e44f4a8b0d294045b6ac98/workflows.tar.gz'}



In [104]:
# Deploy Patch attack on test set.
response_deploy_le_net_patches_mnist_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P patch_deployment_method=corrupt"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

# Get the run ID of the training set.
response_deploy_le_net_patches_mnist_testing = get_run_id(response_deploy_le_net_patches_mnist_testing)

Patch deployment (LeNet-5 architecture) job submitted

{'createdOn': '2021-04-29T04:24:42.332101',
 'dependsOn': '39a44294-3b7e-4fa2-b4f1-3e8ed026884e',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=b81c3b9453e24dc8850d074b43c56b93 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/testing '
                     '-P patch_deployment_method=corrupt',
 'experimentId': 10,
 'jobId': 'd8f4fe18-c8eb-4033-80b4-efe11927c7ae',
 'lastModified': '2021-04-29T04:24:42.332101',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/202d801708204a029092e7e5e87ef714/workflows.tar.gz'}



### Patch Attack Evaluation: Baseline MNIST Model

Now we will run an inference step to check the patch-attacked dataset with our MNIST-trained model.

In [105]:
# Check patched dataset results   
response_le_net_infer_le_net_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)
        
print("Patch evaluation (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

Patch evaluation (LeNet-5 architecture) job submitted

{'createdOn': '2021-04-29T04:24:42.332101',
 'dependsOn': '39a44294-3b7e-4fa2-b4f1-3e8ed026884e',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=b81c3b9453e24dc8850d074b43c56b93 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/testing '
                     '-P patch_deployment_method=corrupt',
 'experimentId': 10,
 'jobId': 'd8f4fe18-c8eb-4033-80b4-efe11927c7ae',
 'lastModified': '2021-04-29T04:24:47.158921',
 'mlflowRunId': '209139b3250b4fd0b88aed48c5545787',
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/202d801708204a029092e7e5e87ef714/workflows.tar.gz'}



In [106]:
# Wait for the job to finish
response_le_net_infer_le_net_patch = wait_until_finished(response_le_net_infer_le_net_patch)

# Check on the patch evaluation results
results = get_mlflow_results(response_le_net_infer_le_net_patch)
print("Baseline model results on adversarially patched dataset: ")
pprint.pprint(results.data.metrics)

Baseline model results on adversarially patched dataset: 
{'accuracy': 0.36518430709838867,
 'auc': 0.7587533593177795,
 'loss': 2.0189787626266478,
 'precision': 0.8056603670120239,
 'recall': 0.29937902092933655}



We can see that the adversarial patch attack causes a noticeable decrease in the model's accuracy scores.

We will now test various defenses against the patch attacked images.

## Defenses: Image Preprocessing and Adversarial Training

The next part of the adversarial patch demo focuses on investigating effective defenses against the attack.

### Preprocessing Defenses: Spatial Smoothing, JPEG Compression, Gaussian Augmentation

Here we will investigate three preprocessing defenses that can be applied over the images before inference.

#### Spatial Smoothing Defense:

Here, we can adjust the `spatial_smoothing_window_size` parameter to increase or decrease the sliding window of the smoothing defense.
Larger values will create more noticeable distortions but can also help mask any adversarial perturbations.

In [111]:
response_le_net_spatial_smoothing_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="spatial_smoothing",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/training",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            "-P spatial_smoothing_window_size=2",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("Spatial smoothing defense job submitted")
print("")
pprint.pprint(response_le_net_spatial_smoothing_test_set)
print("")

response_le_net_spatial_smoothing_test_set = get_run_id(response_le_net_spatial_smoothing_test_set)

Spatial smoothing defense job submitted

{'createdOn': '2021-04-29T05:47:16.142362',
 'dependsOn': 'd8f4fe18-c8eb-4033-80b4-efe11927c7ae',
 'entryPoint': 'spatial_smoothing',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/training -P batch_size=20 -P '
                     'load_dataset_from_mlruns=true -P '
                     'spatial_smoothing_window_size=2 -P '
                     'dataset_run_id=209139b3250b4fd0b88aed48c5545787 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 10,
 'jobId': '8d1208da-6997-49f9-ad9f-93de51501adb',
 'lastModified': '2021-04-29T05:47:16.142362',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/d2d95bd7636e4be9982fe011f8a65d9d/workflows.tar.gz'}



In [112]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_spatial_smoothing_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_spatial_smoothing_test_set['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=spatial_smoothing_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_spatial_smoothing_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_spatial_smoothing_images)
print("")

response_evaluate_spatial_smoothing_images = wait_until_finished(response_evaluate_spatial_smoothing_images)
results = get_mlflow_results(response_evaluate_spatial_smoothing_images)
print("Adversarial Patches with Spatial Smoothing Results: \n")
pprint.pprint(results.data.metrics)

Inference job submitted

{'createdOn': '2021-04-29T05:47:22.166931',
 'dependsOn': '8d1208da-6997-49f9-ad9f-93de51501adb',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=896d2ee3682642dbb75f6696594d8a3d -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P batch_size=20 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 10,
 'jobId': 'a72a31e3-d751-4418-9c6b-49fd99de71b6',
 'lastModified': '2021-04-29T05:47:22.166931',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/41c49f07704348859d423e7f78d20c15/workflows.tar.gz'}

Adversarial Patches with Spatial Smoothing Results: 

{'accuracy': 0.3682364821434021,
 'auc': 0.7625458836555481,
 'loss': 1.9789006578659485,
 'precision': 0.7761310338973999,
 'recall': 0.2990981936454773}


It appears that spatial smoothing does not protect well against adversarial patches.

#### JPEG Compression Defense:

We can adjust the image compression quality by modifying the `jpeg_compression_quality` field. 
Enter any value between 1 (worst) to 95 (best).

In [113]:
response_le_net_jpeg_compression_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jpeg_compression",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir=/{DATASET_DIR}/training",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P jpeg_compression_quality=30",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("JPEG compression defense job submitted")
print("")
pprint.pprint(response_le_net_jpeg_compression_test_set)
print("")

response_le_net_jpeg_compression_test_set = get_run_id(response_le_net_jpeg_compression_test_set)

JPEG compression defense job submitted

{'createdOn': '2021-04-29T05:48:05.856451',
 'dependsOn': 'd8f4fe18-c8eb-4033-80b4-efe11927c7ae',
 'entryPoint': 'jpeg_compression',
 'entryPointKwargs': '-P data_dir=//nfs/data/Mnist/training -P batch_size=20 '
                     '-P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=209139b3250b4fd0b88aed48c5545787 -P '
                     'jpeg_compression_quality=30 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 10,
 'jobId': '2287e970-b585-49b0-9efa-bd0e54b49768',
 'lastModified': '2021-04-29T05:48:05.856451',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/c759fd4188644972bc80bbc001da6f2d/workflows.tar.gz'}



In [114]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_jpeg_compression_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_jpeg_compression_test_set['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=jpeg_compression_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_jpeg_compression_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_jpeg_compression_images)
print("")

response_evaluate_jpeg_compression_images = wait_until_finished(response_evaluate_jpeg_compression_images)
results = get_mlflow_results(response_evaluate_jpeg_compression_images)
print("Adversarial Patches with JPEG Compression Results: \n")
pprint.pprint(results.data.metrics)


Inference job submitted

{'createdOn': '2021-04-29T05:48:10.979385',
 'dependsOn': '2287e970-b585-49b0-9efa-bd0e54b49768',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=fa13446f49a04ad187a28f85841513bc -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P batch_size=20 -P '
                     'adv_tar_name=jpeg_compression_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 10,
 'jobId': '66a6224d-e873-4c39-abbf-3ee5d3a6cc2e',
 'lastModified': '2021-04-29T05:48:10.979385',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/5f5c9a1c713a495ab6142babbf139a9c/workflows.tar.gz'}

Adversarial Patches with JPEG Compression Results: 

{'accuracy': 0.36563125252723694,
 'auc': 0.7546454071998596,
 'loss': 2.0862985856547382,
 'precision': 0.7538230419158936,
 'recall': 0.30130261182785034}


#### Gaussian Augmentation Defense:

We can adjust the amount of noise created by the defense by adjusting the `gaussian_augmentation_sigma` parameter. 
Please enter any positive value for sigma.

In [116]:
response_le_net_gaussian_augmentation_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gaussian_augmentation",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir=/{DATASET_DIR}/training",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            "-P gaussian_augmentation_sigma=0.3"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)


print("Gaussian Augmentation defense job submitted")
print("")
pprint.pprint(response_le_net_gaussian_augmentation_test_set)
print("")

response_le_net_gaussian_augmentation_test_set = get_run_id(response_le_net_gaussian_augmentation_test_set)

Gaussian Augmentation defense job submitted

{'createdOn': '2021-04-29T05:49:14.918148',
 'dependsOn': 'd8f4fe18-c8eb-4033-80b4-efe11927c7ae',
 'entryPoint': 'gaussian_augmentation',
 'entryPointKwargs': '-P data_dir=//nfs/data/Mnist/training -P batch_size=20 '
                     '-P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=209139b3250b4fd0b88aed48c5545787 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset -P '
                     'gaussian_augmentation_sigma=0.3',
 'experimentId': 10,
 'jobId': 'bb8d0a83-0734-48a0-b29b-74f404172841',
 'lastModified': '2021-04-29T05:49:14.918148',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/1840f9ab81874e1e8e6274f2b0804fab/workflows.tar.gz'}



In [117]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_gaussian_augmentation_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_gaussian_augmentation_test_set['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=gaussian_augmentation_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_gaussian_augmentation_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_gaussian_augmentation_images)
print("")

response_evaluate_gaussian_augmentation_images = wait_until_finished(response_evaluate_gaussian_augmentation_images)
results = get_mlflow_results(response_evaluate_gaussian_augmentation_images)
print("Adversarial Patches with Gaussian Augmentation Results: \n")
pprint.pprint(results.data.metrics)


Inference job submitted

{'createdOn': '2021-04-29T05:49:20.049768',
 'dependsOn': 'bb8d0a83-0734-48a0-b29b-74f404172841',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=a3e3b06af9d84f39b37c578f28eede7b -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P batch_size=20 -P '
                     'adv_tar_name=gaussian_augmentation_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 10,
 'jobId': '7d4defb2-ec7d-4732-81c3-9588ac53f143',
 'lastModified': '2021-04-29T05:49:20.049768',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/f05b38b9b67b4208bca24995bceb58b9/workflows.tar.gz'}

Adversarial Patches with Gaussian Augmentation Results: 

{'accuracy': 0.3254508972167969,
 'auc': 0.6982883810997009,
 'loss': 2.709391103001061,
 'precision': 0.43165016174316406,
 'recall': 0.27494990825653076}


It appears that of the three preprocessing defenses, gaussian augmentation might actually improve patch attach effectiveness.

Since preprocessing defenses appear to be ineffective at stopping adversarial patches, let's try another approach.

In [133]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar czf workflows.tar.gz src/gaussian_augmentation.py src/deploy_patch.py src/init_model.py src/defenses_image_preprocessing.py src/tasks.py src/train.py src/import_keras.py src/spatial_smoothing.py src/generate_patch.py src/jpeg_compression.py src/attacks_patch_updated.py src/fgm.py src/infer.py src/registry_art_updated.py src/estimators_keras_classifiers_updated.py src/data_tensorflow_updated.py MLproject
chmod 644 workflows.tar.gz


## Adversarial Training Defense:

Finally, we will train a new copy of the LeNet model on training set that contains adversarial patches.
In doing so, the model learns to ignore the adversarial patches.

In [134]:
# Finally, train and retest patched dataset.

response_deploy_le_net_patches_mnist_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            f"-P dataset_run_id_testing={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P dataset_run_id_training={response_deploy_le_net_patches_mnist_training['mlflowRunId']}",
            "-P batch_size=256",
            "-P model_architecture=le_net",
            f"-P register_model_name={EXPERIMENT_NAME}_adversarial_patch_le_net",
            "-P epochs=30",
            f"-P data_dir_training={DATASET_DIR}/training",
            f"-P data_dir_testing={DATASET_DIR}/testing",
            "-P load_dataset_from_mlruns=True",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_training["jobId"],
)

print("Patch adversarial training (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_adv_training)
print("")

response_deploy_le_net_patches_mnist_adv_training = get_run_id(response_deploy_le_net_patches_mnist_adv_training)

Patch adversarial training (LeNet-5 architecture) job submitted

{'createdOn': '2021-04-29T06:04:39.569265',
 'dependsOn': 'dd0406ef-01c1-4675-aba9-6c8e2965f0d6',
 'entryPoint': 'train',
 'entryPointKwargs': '-P '
                     'dataset_run_id_testing=209139b3250b4fd0b88aed48c5545787 '
                     '-P '
                     'dataset_run_id_training=2b54f7a500c94043a1a3cd696fddf4e6 '
                     '-P batch_size=256 -P model_architecture=le_net -P '
                     'register_model_name=howard_mnist_adversarial_patches_adversarial_patch_le_net '
                     '-P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing -P '
                     'load_dataset_from_mlruns=True',
 'experimentId': 10,
 'jobId': '8acfc0cb-3f74-4fb7-9287-80e27f638b08',
 'lastModified': '2021-04-29T06:04:39.569265',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '2

In [137]:
response_evaluate_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_adversarial_patch_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_adv_training["jobId"],
)

print("Patch evaluation job submitted")
print("")
pprint.pprint(response_evaluate_adv_training)
print("")

response_evaluate_adv_training = wait_until_finished(response_evaluate_adv_training)
results = get_mlflow_results(response_evaluate_adv_training)
print("Adversarial Training Results:")

Patch evaluation job submitted

{'createdOn': '2021-04-29T06:16:42.979016',
 'dependsOn': '8acfc0cb-3f74-4fb7-9287-80e27f638b08',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=209139b3250b4fd0b88aed48c5545787 -P '
                     'model_name=howard_mnist_adversarial_patches_adversarial_patch_le_net '
                     '-P model_version=none -P batch_size=20 -P '
                     'adv_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'adv_data_dir=adv_patch_dataset',
 'experimentId': 10,
 'jobId': 'd6201cca-1a4e-4f6c-abf0-1b3e733346f4',
 'lastModified': '2021-04-29T06:16:42.979016',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/5b404f82e5c14a3c9ed6a7693a1eb84b/workflows.tar.gz'}

Adversarial Training Results:


**We can see that for adversarial patches, adversarial training appears to be a good option.**

**Some other interesting approaches that users may want to investigate by modfifying this demo are:**

-   Generating a separate set of patches for training and test data. 
    -   To do so, add a secondary patch generation job and link the run_id of this new job to one of the existing test/training patch deployment jobs.
-   Experimenting with alternate adversarially trained models. 
    -   If both the MNIST patch demo and MNIST FGM demos are run together, then users can swap the `model={EXPERIMENT_NAME}_adversarial_patch_le_net/None` with the model name of the FGM experiment.
    -   Doing so can let users quickly check which adversarial training offers robust protection against multiple types of attacks. 
     
**Please consult the README documentation for more information regarding available entrypoints and attack/defense parameters.**

<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [ ]:
run_le_net = mlflow_client.get_run(response_le_net_patches["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(run_le_net.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(run_le_net.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(run_le_net.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).